## Backtesting trading strategies

In [2]:
import pandas as pd 
import yfinance as yf
import sys
sys.path.append('../src')
from data_loader.data import load_data
from strategies.macd import MACD
from strategies.stock_trend_following_system import StockTrend
from parallel_processing import ParallelProcess
import talib
import numpy as np
import dask
import multiprocessing
from helper import Profit
from strategies.golden_death_cross import GoldenDeathCross

sys.path.append("../")
from conf import constants
import conf

# instantiate config 
STRATEGY = conf.backtest_conf['strategy']
# instantiate constants
PREPROCESSED_DATAPATH = constants.preprocessed_data_file_path

%load_ext autoreload
%autoreload 2

### EDA on preprocessed dataset
1. Size of data
2. Missing values

In [4]:
file_path = r'C:\Users\ryanl\Desktop\backtesting\data\02_preprocessed\new_preprocessed_data.csv'
pd.read_csv(file_path)

,Date,Unnamed: 0,AIR_Open,AIR_High,AIR_Low,AIR_Close,AIR_Adj Close,AIR_Volume,ABM_Open,ABM_High,...,YUM_Low,YUM_Close,YUM_Adj Close,YUM_Volume,ZBH_Open,ZBH_High,ZBH_Low,ZBH_Close,ZBH_Adj Close,ZBH_Volume
0,2002-01-02,0,9.000000,9.230000,8.990000,9.200000,8.247001,58000,15.650000,15.650000,...,8.599928,8.625090,6.106850,6378570,29.407766,29.514563,29.106796,29.349516,26.646753,818850
1,2002-01-03,1,9.300000,9.680000,9.280000,9.660000,8.659352,122800,15.280000,15.490000,...,8.630482,8.806614,6.235373,5461066,29.223301,29.339806,28.932039,29.165049,26.479280,1058016
2,2002-01-04,2,9.660000,9.900000,9.660000,9.840000,8.820706,110100,15.350000,15.605000,...,8.770669,9.148095,6.477156,8121214,29.145632,29.223301,28.640778,28.689320,26.047354,810198
3,2002-01-07,3,9.850000,9.900000,9.650000,9.700000,8.695210,53500,15.605000,15.605000,...,9.205607,9.318835,6.598044,8309834,28.640778,28.834951,28.466019,28.689320,26.047354,695044
4,2002-01-08,4,9.710000,9.850000,9.650000,9.850000,8.829671,86100,15.200000,15.225000,...,9.435658,9.498562,6.725296,8023288,28.640778,29.650484,28.601942,29.611650,26.884750,898160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5282,2022-12-23,5282,42.299999,42.910000,42.200001,42.889999,42.889999,186200,43.439999,44.310001,...,127.839996,128.899994,127.713448,700800,125.940002,126.769997,125.239998,126.690002,126.238785,789800
5283,2022-12-27,5283,43.500000,44.500000,43.099998,44.310001,44.310001,288100,43.990002,44.740002,...,128.639999,129.899994,128.704224,1300200,127.000000,127.790001,126.169998,127.279999,126.826683,755600
5284,2022-12-28,5284,44.139999,44.389999,43.599998,43.689999,43.689999,187600,44.770000,45.020000,...,129.279999,129.309998,128.119675,964800,127.440002,128.520004,125.779999,125.989998,125.541267,750100
5285,2022-12-29,5285,43.950001,44.700001,43.950001,44.700001,44.700001,201800,44.349998,45.000000,...,128.850006,129.990005,128.793411,875700,126.660004,128.500000,126.139999,127.830002,127.374718,686600


In [12]:
# Load preprocessed data 
PREPROCESSED_DATAPATH = constants.preprocessed_data_file_path
df = load_data(PREPROCESSED_DATAPATH,['_Close'])
df.head()

,Date,AIR_Close,ABM_Close,AEG_Close,MITT_Close,AMN_Close,AOS_Close,ASA_Close,T_Close,ATS_Close,...,WKC_Close,WWE_Close,WOR_Close,XOXO_Close,XRM_Close,XPO_Close,XRX_Close,XYL_Close,YUM_Close,ZBH_Close
0,2002-01-02,9.20,15.380,25.903847,NaN,27.400000,3.191667,6.633333,30.135952,NaN,...,4.5600,13.50,14.00,0.47,NaN,NaN,27.457182,NaN,8.625090,29.349516
1,2002-01-03,9.66,15.320,26.000000,NaN,27.250000,3.268333,6.600000,30.339878,NaN,...,4.5375,13.71,14.10,0.46,NaN,NaN,27.404480,NaN,8.806614,29.165049
2,2002-01-04,9.84,15.605,25.038462,NaN,26.969999,3.380000,6.580000,30.203926,NaN,...,4.5750,13.74,14.45,0.59,NaN,NaN,26.482212,NaN,9.148095,28.689320
3,2002-01-07,9.70,15.300,24.576923,NaN,26.850000,3.361667,6.750000,30.143505,NaN,...,4.5500,13.70,14.57,0.54,NaN,NaN,26.034256,NaN,9.318835,28.689320
4,2002-01-08,9.85,15.020,24.269232,NaN,26.500000,3.216667,6.880000,30.015106,NaN,...,4.6425,13.70,14.08,0.59,NaN,NaN,26.218710,NaN,9.498562,29.611650


In [18]:
# Size of data
num_stocks = len(df.filter(regex='Close').columns.tolist())
print(f'Number of stocks: {num_stocks}')
print(f"Duration of backtesting: 20 years from {df['Date'].iloc[0]} to {df['Date'].iloc[-1]}")

Number of stocks: 1428
Duration of backtesting: 20 years from 2002-01-02 to 2022-12-30


In [7]:
# Top 10 tickers with missing values 
missing_values = df.isnull().any().sum()
print(f'Number of tickers with missing values: {missing_values}\
      \nTop 10 tickers with missing values:')
df.isnull().sum().sort_values(ascending=False)[:10]

Number of tickers with missing values: 934      
Top 10 tickers with missing values:


CPRI_Close    2508
CPRI_Low      2508
CIVI_Close    2508
VTLE_Close    2508
VTLE_Low      2508
CIVI_Low      2508
BUI_Low       2493
BUI_Close     2493
APTV_Close    2489
APTV_Low      2489
dtype: int64

### MACD

In [2]:
# load preprocessed dataset 
PREPROCESSED_DATAPATH = constants.preprocessed_data_file_path
macd_df = load_data(PREPROCESSED_DATAPATH,['_Close'])
macd_df.head()

,Date,AIR_Close,ABM_Close,AEG_Close,MITT_Close,AMN_Close,AOS_Close,ASA_Close,T_Close,ATS_Close,...,WKC_Close,WWE_Close,WOR_Close,XOXO_Close,XRM_Close,XPO_Close,XRX_Close,XYL_Close,YUM_Close,ZBH_Close
0,2002-01-02,9.20,15.380,25.903847,NaN,27.400000,3.191667,6.633333,30.135952,NaN,...,4.5600,13.50,14.00,0.47,NaN,NaN,27.457182,NaN,8.625090,29.349516
1,2002-01-03,9.66,15.320,26.000000,NaN,27.250000,3.268333,6.600000,30.339878,NaN,...,4.5375,13.71,14.10,0.46,NaN,NaN,27.404480,NaN,8.806614,29.165049
2,2002-01-04,9.84,15.605,25.038462,NaN,26.969999,3.380000,6.580000,30.203926,NaN,...,4.5750,13.74,14.45,0.59,NaN,NaN,26.482212,NaN,9.148095,28.689320
3,2002-01-07,9.70,15.300,24.576923,NaN,26.850000,3.361667,6.750000,30.143505,NaN,...,4.5500,13.70,14.57,0.54,NaN,NaN,26.034256,NaN,9.318835,28.689320
4,2002-01-08,9.85,15.020,24.269232,NaN,26.500000,3.216667,6.880000,30.015106,NaN,...,4.6425,13.70,14.08,0.59,NaN,NaN,26.218710,NaN,9.498562,29.611650


In [3]:
parallel_processing = ParallelProcess(macd_df,STRATEGY)
parallel_processing.run_pipeline()

Complete parallel processing
Complete parallel processing
Complete parallel processing
Complete parallel processing
Complete parallel processing
Complete parallel processing
Complete parallel processing
Complete parallel processing
Complete parallel processing
Complete parallel processing
Complete parallel processing


### Profits for macd strategy

In [2]:
macd_df = pd.read_csv('../data/03_strategy_positions/macd_positions.csv',index_col=0)
macd_df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_6296\1209835307.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,

,AAIC_Position,AAIC_Entry,AAIC_Exit,AAP_Position,AAP_Entry,AAP_Exit,AAT_Position,AAT_Entry,AAT_Exit,AA_Position,...,X_Exit,YUM_Position,YUM_Entry,YUM_Exit,ZBH_Position,ZBH_Entry,ZBH_Exit,ZTR_Position,ZTR_Entry,ZTR_Exit
0,long,2002-03-11,2002-03-19,long,2002-03-19,2002-03-20,long,2011-03-21,2011-03-22,long,...,2002-04-01,long,2002-02-26,2002-02-27,long,2002-04-29,2002-05-01,long,2002-03-06,2002-03-07
1,long,2002-03-21,2002-03-22,long,2002-06-20,2002-06-21,long,2011-04-25,2011-04-26,long,...,2002-05-09,long,2002-04-10,2002-04-11,long,2002-06-20,2002-06-28,long,2002-03-21,2002-03-26
2,long,2002-04-29,2002-05-01,long,2002-07-29,2002-07-30,long,2011-05-11,2011-05-12,long,...,2002-08-02,long,2002-04-23,2002-04-24,long,2002-07-26,2002-07-29,long,2002-03-26,2002-03-28
3,long,2002-06-13,2002-06-14,long,2002-09-12,2002-09-17,long,2011-06-01,2011-06-15,long,...,2002-08-21,long,2002-05-16,2002-05-20,long,2002-09-09,2002-09-19,long,2002-04-15,2002-04-17
4,long,2002-08-01,2002-08-07,long,2002-09-30,2002-10-03,long,2011-06-22,2011-06-23,long,...,2002-08-21,long,2002-06-11,2002-06-19,long,2002-09-23,2002-09-24,long,2002-07-08,2002-07-18


In [5]:
test_df = macd_df.iloc[:,:3]
test_df.head()
# test_df.columns[test_df.columns.str.endswith('_Entry')][0]

,AAIC_Position,AAIC_Entry,AAIC_Exit
0,long,2002-03-11,2002-03-19
1,long,2002-03-21,2002-03-22
2,long,2002-04-29,2002-05-01
3,long,2002-06-13,2002-06-14
4,long,2002-08-01,2002-08-07


In [3]:
close_df = load_data(PREPROCESSED_DATAPATH,['_Close'])
close_df.head()

,Date,AIR_Close,ABM_Close,AEG_Close,MITT_Close,AMN_Close,AOS_Close,ASA_Close,T_Close,ATS_Close,...,WKC_Close,WWE_Close,WOR_Close,XOXO_Close,XRM_Close,XPO_Close,XRX_Close,XYL_Close,YUM_Close,ZBH_Close
0,2002-01-02,9.20,15.380,25.903847,NaN,27.400000,3.191667,6.633333,30.135952,NaN,...,4.5600,13.50,14.00,0.47,NaN,NaN,27.457182,NaN,8.625090,29.349516
1,2002-01-03,9.66,15.320,26.000000,NaN,27.250000,3.268333,6.600000,30.339878,NaN,...,4.5375,13.71,14.10,0.46,NaN,NaN,27.404480,NaN,8.806614,29.165049
2,2002-01-04,9.84,15.605,25.038462,NaN,26.969999,3.380000,6.580000,30.203926,NaN,...,4.5750,13.74,14.45,0.59,NaN,NaN,26.482212,NaN,9.148095,28.689320
3,2002-01-07,9.70,15.300,24.576923,NaN,26.850000,3.361667,6.750000,30.143505,NaN,...,4.5500,13.70,14.57,0.54,NaN,NaN,26.034256,NaN,9.318835,28.689320
4,2002-01-08,9.85,15.020,24.269232,NaN,26.500000,3.216667,6.880000,30.015106,NaN,...,4.6425,13.70,14.08,0.59,NaN,NaN,26.218710,NaN,9.498562,29.611650


In [6]:
profit = Profit(test_df,close_df)
profit_df = profit.calculate_profit('long')
profit.calculate_long_term_profit(profit_df,start_year=2022,end_year=2022)

1.2265999620167967

In [13]:
for i in range(0, len(macd_df.columns), 3):
    triple_cols = macd_df.iloc[:,i:i+3]
    # filter for 'long' positions 
    triple_cols = triple_cols[triple_cols[triple_cols.columns[0]]=='long']
    profit = Profit(triple_cols,close_df)
    profit_df = profit.calculate_profit('long')
    result = profit.calculate_long_term_profit(profit_df,start_year=2022,end_year=2022)
    print(f"Result for {triple_cols.columns[0]}: {result}")

Result for AAIC_Position: 1.1198363139808591
Result for AAP_Position: 0.6594841635182145
Result for AAT_Position: 0.9993166936662261
Result for AA_Position: 1.1534900102276269
Result for ABB_Position: 0.854457547562928
Result for ABC_Position: 1.1097413365938769
Result for ABEV_Position: 1.0020575816869637
Result for ABG_Position: 0.9855805003758561
Result for ABM_Position: 1.1278202751686726
Result for ABR_Position: 0.8645935998124507
Result for ABT_Position: 0.9501003802896969
Result for AB_Position: 1.127134692046016
Result for ACCO_Position: 1.0297808440692906
Result for ACM_Position: 0.9441458572528725
Result for ACN_Position: 1.0303446421549995
Result for ACR_Position: 1.0571689273152949
Result for ADC_Position: 1.137575948282623
Result for ADM_Position: 1.2342014780437285
Result for ADX_Position: 1.099239484101466
Result for AEE_Position: 1.0597537379868864
Result for AEG_Position: 1.0845753222110057
Result for AEL_Position: 0.8549058293928365
Result for AEM_Position: 1.05131966

Result for CATO_Position: 0.9413399555676196
Result for CAT_Position: 1.0948114539889497
Result for CBRE_Position: 1.1254547490182971
Result for CBT_Position: 0.9582686315345843
Result for CBU_Position: 0.9656577847690438
Result for CBZ_Position: 0.9226770363348911
Result for CB_Position: 0.9156042271321175
Result for CCEP_Position: 1.2134051752281958
Result for CCI_Position: 1.0766627710164172
Result for CCJ_Position: 1.0697996490051576
Result for CCK_Position: 1.2796992282747162
Result for CCL_Position: 0.978030178716422
Result for CCO_Position: 0.7680317015596894
Result for CCRD_Position: 1.098241914381912
Result for CDE_Position: 0.9981363110582251
Result for CEE_Position: 0.9927403351872242
Result for CEM_Position: 1.062368215154563
Result for CEQP_Position: 0.9108038705412703
Result for CE_Position: 1.063575617661076
Result for CFR_Position: 1.0477056549635506
Result for CF_Position: 1.2617541005262998
Result for CGAU_Position: 0.9472595820206874
Result for CGA_Position: 0.990393

In [2]:
# load preprocessed dataset 
PREPROCESSED_DATAPATH = conf.backtest_conf["data"]["preprocessed_datapath"]
df = load_data(PREPROCESSED_DATAPATH,['_Close','_Low'])
df.head()

,Date,AIR_Low,AIR_Close,ABM_Low,ABM_Close,AEG_Low,AEG_Close,MITT_Low,MITT_Close,AMN_Low,...,XPO_Low,XPO_Close,XRX_Low,XRX_Close,XYL_Low,XYL_Close,YUM_Low,YUM_Close,ZBH_Low,ZBH_Close
0,2002-01-02,8.99,9.20,15.050,15.380,25.653847,25.903847,NaN,NaN,26.600000,...,NaN,NaN,26.666668,27.457182,NaN,NaN,8.599928,8.625090,29.106796,29.349516
1,2002-01-03,9.28,9.66,15.125,15.320,25.721153,26.000000,NaN,NaN,27.100000,...,NaN,NaN,27.009224,27.404480,NaN,NaN,8.630482,8.806614,28.932039,29.165049
2,2002-01-04,9.66,9.84,15.250,15.605,24.692308,25.038462,NaN,NaN,26.500000,...,NaN,NaN,25.639000,26.482212,NaN,NaN,8.770669,9.148095,28.640778,28.689320
3,2002-01-07,9.65,9.70,15.300,15.300,24.355768,24.576923,NaN,NaN,26.850000,...,NaN,NaN,23.978920,26.034256,NaN,NaN,9.205607,9.318835,28.466019,28.689320
4,2002-01-08,9.65,9.85,14.915,15.020,23.884615,24.269232,NaN,NaN,26.450001,...,NaN,NaN,25.032938,26.218710,NaN,NaN,9.435658,9.498562,28.601942,29.611650


### Parallel processing

In [5]:
stock_trend_df = pd.read_csv(r'C:\Users\User\Desktop\side projects\backtesting\data\intermediate\stocktrend_positions.csv')
stock_trend_df.loc[stock_trend_df['AIR_entry_date'].notnull()]

C:\Users\User\AppData\Local\Temp\ipykernel_19088\3624734702.py:1: DtypeWarning: Columns (3,4,5,6,7,8,11,12,13,14,15,16,17,18,21,22,23,24,25,26,27,28,29,30,33,34,35,36,37,38,39,40,43,44,45,46,47,48,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,133,134,135,136,137,138,139,140,141,142,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,183,184,185,186,187,188,189,190,191,192,195,196,197,198,199,200,201,202,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,279,280,281,282,283,284,

,Unnamed: 0,AIR_entry_date,AIR_exit_date,ABM_entry_date,ABM_exit_date,AEG_entry_date,AEG_exit_date,MITT_entry_date,MITT_exit_date,AMN_entry_date,...,XPO_entry_date,XPO_exit_date,XRX_entry_date,XRX_exit_date,XYL_entry_date,XYL_exit_date,YUM_entry_date,YUM_exit_date,ZBH_entry_date,ZBH_exit_date
118,118,2002-06-24,2002-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,120,2002-06-25,2002-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,121,2002-06-26,2002-06-28,NaN,NaN,NaN,NaN,NaN,NaN,2002-06-26,...,NaN,NaN,NaN,NaN,NaN,NaN,2002-06-28,2002-07-08,NaN,NaN
122,122,2002-06-27,2002-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2002-06-28,2002-07-08,NaN,NaN
123,123,2002-07-01,2002-07-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5148,5148,2022-06-14,2022-06-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5149,5149,2022-06-16,2022-06-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5276,5276,2022-12-15,2022-12-19,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5278,5278,2022-12-21,2022-12-27,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# parallel_processing = ParallelProcess(df,STRATEGY)
# parallel_processing.run_pipeline()

### stocktrend profits

In [12]:
stocktrend_df = pd.read_csv(r'C:\Users\User\Desktop\side projects\backtesting\data\03_strategy_positions\stocktrend_positions.csv')
stocktrend_df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_2884\2396964136.py:1: DtypeWarning: Columns (3,4,5,6,7,8,11,12,13,14,15,16,17,18,21,22,23,24,25,26,27,28,29,30,33,34,35,36,37,38,39,40,43,44,45,46,47,48,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,133,134,135,136,137,138,139,140,141,142,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,183,184,185,186,187,188,189,190,191,192,195,196,197,198,199,200,201,202,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,279,280,281,282,283,284,2

,Unnamed: 0,AIR_entry_date,AIR_exit_date,ABM_entry_date,ABM_exit_date,AEG_entry_date,AEG_exit_date,MITT_entry_date,MITT_exit_date,AMN_entry_date,...,XPO_entry_date,XPO_exit_date,XRX_entry_date,XRX_exit_date,XYL_entry_date,XYL_exit_date,YUM_entry_date,YUM_exit_date,ZBH_entry_date,ZBH_exit_date
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Golden death cross

In [7]:
golden_death_cross = GoldenDeathCross(df)
output = golden_death_cross.get_cross(df['A_Close'])
output[output['cross']=='golden']

,Date,cross,long_term_ma
351,2003-05-27,golden,10.832940
880,2005-06-30,golden,16.508977
1243,2006-12-07,golden,24.061051
1632,2008-06-26,golden,24.671781
1890,2009-07-07,golden,13.468348
2223,2010-10-29,golden,22.533691
2556,2012-02-27,golden,28.641774
2778,2013-01-15,golden,28.082582
3268,2014-12-24,golden,40.388131
3333,2015-03-31,golden,40.602784
